In [27]:
# 자동 미분 및 그래디언트
# 자동 미분은 신경망 학습을 위한 역전파와 같은 머신러닝 알고리즘을 구현하는 데 유용


# 그래디언트 계산
# 자동 미분을 위해 TensorFlow는 정방향 패스 동안 어떤 연산이 어떤 순서로 발생하는지 기억
# 그 후, 역방향 패스 동안 TensorFlow는 이 연산 목록을 역순으로 이동하여 그래디언트 계산


# 그래디언트 테이프 : TensorFlow에서 자동 미분을 위해 제공되는 API
# tf.GradientTape는 컨텍스트 안에서 실행된 모든 연산을 테이프에 기록
# 그 다음, TensorFlow는 후진 방식 자동 미분을 사용해 테이프에 기록된 연산의 그래디언트 계산
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# x = tf.Variable(3.0)
# with tf.GradientTape() as tape:
#     y = x ** 2

# 일부 연산을 기록한 후, GradientTape.gradient(target, source)를 사용해 그래디언트 계산
# dy_dx = tape.gradient(y, x)
# dy_dx.numpy()

# w = tf.Variable(tf.random.normal((3, 2)), name='w')
# b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
# x = [[1., 2., 3.]]

# with tf.GradientTape() as tape:
#     y = x @ w + b
#     loss = tf.reduce_mean(y ** 2)

# [dl_dw, dl_db] = tape.gradient(loss, [w, b])

# print(w.shape)
# print(dl_dw.shape)

# my_vars = {
#     'w': w,
#     'b': b
# }

# grad = tape.gradient(loss, my_vars)
# grad['b']


# 모델에 대한 그래디언트
# tf.Module의 모든 서브 클래스는 Module.trainable_variables 속성에서 변수를 집계하므로 간단하게 그래디언트 계산 가능
# layer = tf.keras.layers.Dense(2, activation='relu')
# x = tf.constant([[1., 2., 3.]])

# with tf.GradientTape() as tape:
#     # Forward pass
#     y = layer(x)
#     loss = tf.reduce_mean(y ** 2)

# Calculate gradients with respect to every trainable variable
# grad = tape.gradient(loss, layer.trainable_variables)
# for var, g in zip(layer.trainable_variables, grad):
#     print(f'{var.name}, shape: {g.shape}')


# 테이프의 감시 대상 제어
# 테이프는 역방향 패스의 그래디언트를 계산하기 위해 정방향 패스에 기록할 연산을 알아야 함
# 테이프는 중간 출력에 대한 참조를 보유하므로 불필요한 연산을 기록하지 않음
# 가장 일반적인 사용 사례는 모든 모델의 훈련 가능한 변수에 대해 손실의 그래디언트를 계산
# x0 = tf.Variable(3.0, name='x0')
# x1 = tf.Variable(3.0, name='x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor
# x2 = tf.Variable(2.0, name='x2') + 1.0
# x3 = tf.constant(3.0, name='x3')

# with tf.GradientTape() as tape:
#     y = (x0 ** 2)  + (x1 ** 2) + (x2 ** 2)

# grad = tape.gradient(y, [x0, x1, x2, x3])
# for g in grad:
#     print(g)

# GradientTape.watched_variables 메서드를 사용해 테이프에서 감시 중인 변수 나열
# [var.name for var in tape.watched_variables()]

# tf.Tensor에 대한 그래디언트를 기록하려면 GradientTape.watch(x) 호출
# x = tf.constant(3.0)
# with tf.GradientTape() as tape:
#     tape.watch(x)
#     y = x ** 2

# dy_dx = tape.gradient(y, x)
# print(dy_dx.numpy())

# tf.Variable을 감시하는 기본 동작을 비활성화하려면, 그래디언트 테이프를 만들 때 watch_accessed_variables=False를 설정
# x0 = tf.Variable(0.0)
# x1 = tf.Variable(10.0)

# with tf.GradientTape(watch_accessed_variables=False) as tape:
#     tape.watch(x1)
#     y0 = tf.math.sin(x0)
#     y1 = tf.nn.softplus(x1)
#     y = y0 + y1
#     ys = tf.reduce_sum(y)

# grad = tape.gradient(ys, {'x0': x0, 'x1': x1})
# print("dy/dx0:", grad['x0'])
# print("dy/dx1:", grad['x1'].numpy())


# 중간 결과
# tf.GradientTape 컨텍스트 내에서 계산된 중간값과 관련해 출력의 그래디언트 요청 가능
# x = tf.constant(3.0)
# with tf.GradientTape() as tape:
#     tape.watch(x)
#     y = x * x
#     z = y * y

# print(tape.gradient(z, y).numpy())

# 동일한 계산에 대해 여러 그래디언트를 계산하려면 persistent=True 그래디언트 테이프를 생성
# 이러면 테이프 객체가 가비지 수집될 때 리소스가 해제되면 gradient 메서드를 여러 번 호출 가능
# x = tf.constant([1, 3.0])
# with tf.GradientTape(persistent=True) as tape:
#     tape.watch(x)
#     y = x * x
#     z = y * y

# print(tape.gradient(z, x).numpy())
# print(tape.gradient(y, x).numpy())

# Drop the reference to the tape
# del tape


# 성능에 대한 참고 사항
# 그래디언트 테이프 컨텍스트 내에서 연산을 수행하는 것과 관련해 작은 오버헤드가 있음
# 대부분의 Eager 실행에는 상당한 비용이 들지 않지만, 필요한 경우에만 테이프 컨텍스트를 사용해야 함
# 그래디언트 테이프는 메모리를 사용하여 역방향 패스 동안 사용하기 위해 입력 및 출력을 포함한 중간 결과 저장
# 효율성을 위해 일부 연산은 중간 결과를 유지할 필요가 없으며 정방향 패스 동안 정리됨
# 그러나, 테이프에서 persistent=True를 사용하면 아무것도 삭제되지 않으며 최대 메모리 사용량이 높아짐


# 스칼라가 아닌 대상의 그래디언트
# x = tf.Variable(2.0)
# with tf.GradientTape(persistent=True) as tape:
#     y0 = x ** 2
#     y1 = 1 / x

# print(tape.gradient(y0, x).numpy())
# print(tape.gradient(y1, x).numpy())
# print(tape.gradient({'y0': y0, 'y1': y1}, x).numpy())

# x = tf.Variable(2.)
# with tf.GradientTape() as tape:
#     y = x * [3., 4.]

# print(tape.gradient(y, x).numpy())

# 요소별 계산의 경우, 각 요소가 독립적이므로 합의 그래디언트는 입력 요소와 관련해 각 요소의 미분 제공
# x = tf.linspace(-10.0, 10.0, 200 + 1)
# with tf.GradientTape() as tape:
#     tape.watch(x)
#     y = tf.nn.sigmoid(x)

# dy_dx = tape.gradient(y, x)

# plt.plot(x, y, label='y')
# plt.plot(x, dy_dx, label='dy/dx')
# plt.legend()
# _ = plt.xlabel('x')


# 흐름 제어
# x = tf.constant(1.0)
# v0 = tf.Variable(2.0)
# v1 = tf.Variable(2.0)

# with tf.GradientTape(persistent=True) as tape:
#     tape.watch(x)
#     if x > 0.0:
#         result = v0
#     else:
#         result = v1 ** 2

# dv0, dv1 = tape.gradient(result, [v0, v1])
# print(dv0)
# print(dv1)

# dx = tape.gradient(result, x)
# print(dx)


# Gradient가 None을 반환하는 경우
# target이 source와 연결되어 있지 않으면 gradient(target, source)가 None을 반환
# x = tf.Variable(2.)
# y = tf.Variable(3.)

# with tf.GradientTape() as tape:
#     z = y * y
# print(tape.gradient(z, x))

## 1. 변수를 텐서로 대체
# 테이프의 감시 대상 제어 섹션에서 테이프가 자동으로 tf.Variable을 감시하지만 tf.Tensor는 감시하지 않음
# 한 가지 일반적인 오류는 Variable.assign를 사용해 tf.Variable를 업데이트하는 대신 실수로 tf.Tensor로 대체하는 것
# x = tf.Variable(2.0)

# for epoch in range(2):
#     with tf.GradientTape() as tape:
#         y = x + 1
#     print(f"{type(x).__name__}: {tape.gradient(y, x)}")
#     x = x + 1                  # This should be 'x.assign_add(1)'

## 2. TensorFlow 외부에서 계산
# 계산에서 TensorFlow를 종료하면 테이프가 그래디언트 경로를 기록할 수 없음
# x = tf.Variable([[1.0, 2.0],
#                  [3.0, 4.0]], dtype=tf.float32)

# with tf.GradientTape() as tape:
#     x2 = x ** 2

#     # This step is calculated with NumPy
#     y = np.mean(x2, axis=0)

#     # Like most ops, reduce_mean will cast the NumPy array to a constant tensor using 'tf.convert_to_tensor'
#     y = tf.reduce_mean(y, axis=0)

# print(tape.gradient(y, x))

## 3. 정수 또는 문자열을 통해 그래디언트 계산
# 정수와 문자열은 구분할 수 없으며, 계산 경로에서 이러한 데이터 유형을 사용하면 그래디언트는 없음
# dtype을 지정하지 않으면 실수로 int 상수 또는 변수를 만들기 쉬움
# x = tf.constant(10)
# with tf.GradientTape() as tape:
#     tape.watch(x)
#     y = x * x

# print(tape.gradient(y, x))

## 4. 상태 저장 개체를 통해 그래디언트 계산
# 상태가 그래디언트를 중지함
# 상태 저장 객체에서 읽을 때 테이프는 현재 상태만 볼 수 있으며 현재 상태에 이르게 된 기록은 볼 수 없음
# tf.Tensor는 작성된 후에는 변경할 수 없음. 즉, 값은 있지만 상태는 없음
# tf.Variable은 내부 상태와 값을 가지며, 변수를 사용하면 상태를 읽음
# 변수를 사용하여 그래디언트를 계산하는 것이 일반적이지만, 변수의 상태는 그래디언트 계산이 더 멀리 돌아가지 않도록 차단
# 마찬가지로, tf.data.Dataset 반복기와 tf.queue는 상태 저장이며 이들을 통과하는 텐서의 모든 그래디언트는 중지
# x0 = tf.Variable(3.0)
# x1 = tf.Variable(0.0)

# with tf.GradientTape() as tape:
#     # Update x1 = x1 + x0
#     x1.assign_add(x0)
#     # The tape starts recording from x1
#     y = x1 ** 2

# print(tape.gradient(y, x0))


# 그래디언트가 등록되지 않음
# 일부 tf.Operation는 미분 불가능한 것으로 등록되어 None 반환
# 그래디언트가 등록되지 않은 float op를 통해 그래디언트를 얻고자 시도하면 테이프가 자동으로 None을 반환하는 대신 오류 발생
# image = tf.Variable([[[0.5, 0.0, 0.0]]])
# delta = tf.Variable(0.1)

# with tf.GradientTape() as tape:
#     new_image = tf.image.adjust_contrast(image, delta)

# try:
#     print(tape.gradient(new_image, [image, delta]))
#     assert False
# except LookupError as e:
#     print(f"{type(e).__name__}: {e}")


# None 대신 0
# 연결되지 않은 그래디언트의 경우 None 대신 0을 가져오는 것이 편리한 경우가 존재
# 연결되지 않은 그래디언트가 있을 때, unconnected_gradients 인수를 사용해 반환할 항목 결정 가능
x = tf.Variable([2., 2.])
y = tf.Variable(3.)

with tf.GradientTape() as tape:
    z = y ** 2
print(tape.gradient(z, x, unconnected_gradients=tf.UnconnectedGradients.ZERO))

tf.Tensor([0. 0.], shape=(2,), dtype=float32)
